# The extraction of last column of the each patients for the base visit.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import functools

In [3]:
#!pip install dask[complete];
#!pip install --upgrade pandas "dask[complete]"
#python -m pip install "dask[dataframe]" --upgrade

In [12]:
import dask

In [2]:
import dask.dataframe as dd

 We put the main path to the folder where 'counts' files are there.


In [4]:
path1=Path("/scratch/znazari/PPMI_ver_sep2022/RNA_Seq_data/star_ir2/counts/")

 We read the file: in the bl.txt is written the name of all files.

In [5]:
bl_files=pd.read_csv(path1/"bl.txt", header=None )

In [8]:
bl_files.iloc[:,-1:]

,0
0,PPMI-Phase1-IR2.10874.BL.PP0047-6931.5104-SL-0...
1,PPMI-Phase1-IR2.12499.BL.PP0042-3049.5104-SL-1...
2,PPMI-Phase1-IR2.12593.BL.0003149216.5104-SL-13...
3,PPMI-Phase1-IR2.13039.BL.PP0048-8378.5104-SL-1...
4,PPMI-Phase1-IR2.13424.BL.0003180024.5104-SL-18...
...,...
1532,PPMI-Phase2-IR2.65003.BL.0003195560.5104-SL-03...
1533,PPMI-Phase2-IR2.65005.BL.0003451950.5104-SL-26...
1534,PPMI-Phase2-IR2.65008.BL.PP0041-9827.5104-SL-2...
1535,PPMI-Phase2-IR2.70239.BL.0003414527.5104-SL-39...


In [5]:
len(bl_files)

1537

 We define a function where we can take the second phrase seperated by dot. The second phrase 
is the patient ID. So with this functin we want to get the patient IDs from their file's name

In [11]:
def function_names(fname):
    tokens=fname.split('.')
    return tokens[1] 

In [39]:
def merge(a,b):
    a.iloc[:,-1:].reset_index(drop=False).merge(b.iloc[:,-1:].reset_index(drop=True), left_index=True, right_index=True)
    return a

In [38]:
def merge_df(a,b):
    return a.reset_index(drop=True).merge(b.iloc[:,-1:].reset_index(drop=True),left_index=True, right_index=True)

 here we go through all file names and extract their patient IDs and put into a list
 So here we have the list of patient IDs for those whom they have put the data for their base visit

In [12]:
bl_list=[]
for i in range(len(bl_files)):
    bl_list=bl_list+[function_names(bl_files.iloc[i][0])]

 here we read all the files with with base visit from the count folder where we have all the files
 for all the patients and all the visit.
 here we have a list of all files with base visit.

In [23]:
%%time
list_bl_files=[]
for i in range(len(bl_files)):
    list_bl_files=list_bl_files+[dd.read_csv(path1/bl_files.iloc[i][0],skiprows=1,delimiter='\t')]

CPU times: user 7.47 s, sys: 2.28 s, total: 9.74 s
Wall time: 12.5 s


In [14]:
%%time
dfSave = pd.DataFrame()
for i in range(len(bl_files)):
    print(i)
    df = pd.read_csv(path1/bl_files.iloc[i][0],skiprows=1,delimiter='\t')
    
    dfSave[bl_list[i]] = df.iloc[:,-1:]
    
    
    del df
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


<timed exec>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


101


<timed exec>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


102


...

<timed exec>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


1535


<timed exec>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


1536
CPU times: user 13min 12s, sys: 1min 33s, total: 14min 45s
Wall time: 20min 38s


<timed exec>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


In [15]:
dfSave

,10874,12499,12593,13039,13424,14281,14331,14426,15761,16580,...,60024,60036,60095,60171,65002,65003,65005,65008,70239,85242
0,10,10,12,7,10,5,7,4,4,4,...,8,11,2,1,5,6,4,9,5,6
1,192,418,276,252,292,235,162,209,128,99,...,210,512,211,134,168,235,270,188,182,432
2,0,0,2,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57815,346,142,169,158,138,241,65,83,493,69,...,86,132,59,142,50,93,45,802,86,92
57816,48,17,28,24,24,54,3,7,85,10,...,16,13,12,13,5,16,6,129,8,4
57817,766,1506,405,379,1316,581,501,328,517,192,...,1049,1260,494,402,147,872,508,1949,540,2501
57818,62,54,16,67,130,60,63,72,62,105,...,56,73,34,54,117,27,14,82,98,26


In [ ]:
dfSave.to_csv('files.csv', index=False)

In [24]:
list_bl_files[0]

,Geneid,Chr,Start,End,Strand,Length,bams/inQueue_PPMI-Phase1-IR1.10874.BL.PP0047-6931.5104-SL-0659.longRNA-NEBKAP.star.bam
npartitions=1,,,,,,,
,object,object,object,object,object,int64,int64
,...,...,...,...,...,...,...


In [25]:
# here  we take the last column of each file? 
for i in range(len(bl_list)):
    list_bl_files[i].columns = list(list_bl_files[i].columns[:-1])+[bl_list[i]]

In [26]:
list_bl_files[1]

,Geneid,Chr,Start,End,Strand,Length,12499
npartitions=1,,,,,,,
,object,object,object,object,object,int64,int64
,...,...,...,...,...,...,...


In [27]:
# first two columns we merege 

first_two_elements_merge_bl=list_bl_files[0].compute().iloc[:,-1:].reset_index(drop=False).merge(list_bl_files[1].compute().iloc[:,-1:].reset_index(drop=True), left_index=True, right_index=True)

In [28]:
dd_first_two_elements_merge_bl = dd.from_pandas(first_two_elements_merge_bl, npartitions=3)

In [29]:
list_bl_files_conv=[dd_first_two_elements_merge_bl]+list_bl_files[2:]

del list_bl_files

In [30]:
list_bl_files_conv

[Dask DataFrame Structure:
                index  10874  12499
 npartitions=3                     
 0              int64  int64  int64
 19274            ...    ...    ...
 38548            ...    ...    ...
 57819            ...    ...    ...
 Dask Name: from_pandas, 3 tasks,
 Dask DataFrame Structure:
                Geneid     Chr   Start     End  Strand Length  12593
 npartitions=1                                                      
                object  object  object  object  object  int64  int64
                   ...     ...     ...     ...     ...    ...    ...
 Dask Name: rename, 2 tasks,
 Dask DataFrame Structure:
                Geneid     Chr   Start     End  Strand Length  13039
 npartitions=1                                                      
                object  object  object  object  object  int64  int64
                   ...     ...     ...     ...     ...    ...    ...
 Dask Name: rename, 2 tasks,
 Dask DataFrame Structure:
                Geneid     Chr  

In [40]:
merged_bl=functools.reduce(merge_df,list_bl_files_conv)
del list_bl_files_conv


KeyboardInterrupt: 

In [ ]:
merged_bl

In [ ]:
merged_bl.to_csv("random_test_bl_matrix_counts_ir2.txt",  index=False)